<a href="https://colab.research.google.com/github/shirleyzz/cap5610_machineleanrning/blob/master/HW1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#using logistic regression with mean squared error loss
from keras.datasets import mnist
import numpy as np
from keras import backend as K
import keras
from keras.utils import to_categorical

In [0]:
(x_train_origin, y_train_origin),(x_test_origin, y_test_origin) = mnist.load_data()

In [0]:
#classifier of flag, equals target number
def convert_labels(df,flag=0):
  for item in range(df.shape[0]):
    if df[item] == flag:
      df[item] = 1
    else:
      df[item] = 0
  return df

In [0]:
from imblearn.over_sampling import RandomOverSampler
def preprocessing(flag=0,oversampling=False):
  x_train = x_train_origin.reshape((60000, 28 * 28))
  x_test = x_test_origin.reshape((10000, 28 * 28))
  #set the flag to be the one classifier you choose
  train_labels = convert_labels(y_train_origin,flag)
  test_labels = convert_labels(y_test_origin,flag)
  #set oversampling
  if(oversampling):
    ros = RandomOverSampler(random_state=3)
    x_train, train_labels = ros.fit_resample(x_train, train_labels)
  print(x_train.shape)
  train_labels = train_labels.reshape(1,train_labels.shape[0])
  test_labels = test_labels.reshape(1,test_labels.shape[0])
  x_train = x_train.reshape(x_train.shape[0],-1).T
  x_test = x_test.reshape(x_test.shape[0],-1).T
  x_train = x_train.astype('float32') / 255.0
  x_test = x_test.astype('float32') / 255.0
  print(x_train.shape)
  #check whether labels are good
  print(len(train_labels[train_labels==1]))
  return x_train,train_labels,x_test,test_labels

Each of the ten classifiers has an input layer consisting of 28 x 28 input neurons and an output layer consisting of a single output neuron.





In [0]:
def sigmoid(x):
  z = 1.0/(1.0+np.exp(-x))
  return z

In [0]:
def w_b_initialize(dim):
    w = np.zeros((dim,1))
    b = 0  
    return w, b

In [0]:
def propagate(w, b, X, Y):
    m = X.shape[1]
    A = sigmoid(np.dot(w.T, X) + b)
    loss = (1/m) * np.sum(np.square(Y - A))
   
    dL = A - Y
    dA = A * (1 - A) * dL
    
    dw = (1/m)*np.dot(X,dA.T)
    db = 1/m*np.sum(dA)
  
    grads = {"dw": dw,
             "db": db}
    return grads, loss

In [0]:
def optimize(w, b, X, Y, num_iters, batch_size, learning_rate, print_cost):
    costs = []
    n_samples = X.shape[1]
    for i in range(num_iters):
        cost_batch = []
        shuffled_indices = np.random.permutation(n_samples)
        X_shuffled = X[:,shuffled_indices]
        y_shuffled = Y[:,shuffled_indices]
        for j in range(0, n_samples, batch_size):
            x_batch = X_shuffled[:, j:j+batch_size]
            y_batch = y_shuffled[:, j:j+batch_size]
            grads, cost = propagate(w, b, x_batch, y_batch)
            dw = grads["dw"]
            db = grads["db"]
            w = w - learning_rate *  dw    
            b = b - learning_rate *  db
            cost_batch.append(cost)
            cost = np.mean(cost_batch)
        costs.append(cost)            
        if print_cost:
            print ("Cost after iteration %i: %f" % (i+1, cost))
    
    params = {"w": w,
              "b": b}
    
    grads = {"dw": dw,
             "db": db}
    
    return params, grads, costs

In [0]:
def predict(w, b, X):
    m = X.shape[1]
    Y_pred = np.zeros((1, m))
    w = w.reshape(X.shape[0], 1)
    A = sigmoid(np.dot(w.T, X) + b)
    for i in range(A.shape[0]):
        if A[0][i] <= 0.5:
            Y_pred[0, i] = 0
        else:
            Y_pred[0, i] = 1
    return Y_pred

In [0]:
def model(X_train, Y_train, X_test, Y_test, batch, num_iters=20000, learning_rate=0.5, print_cost=False):
    # initialize parameters
    w, b = w_b_initialize(X_train.shape[0])                          
    # stochastic gradient descent 
    parameters, grads, costs = optimize(w, b, X_train, Y_train,num_iters, batch, learning_rate, print_cost)     
    w = parameters["w"]
    b = parameters["b"]
    Y_pred_test = predict(w, b, X_test)
    Y_pred_train = predict(w, b, X_train)
    print("")
    train_accuracy =  100 - np.mean(np.abs(Y_pred_train - Y_train)) * 100
    test_accuracy = 100 - np.mean(np.abs(Y_pred_test - Y_test)) * 100
    print("train accuracy: {} %".format(train_accuracy))
    print("test accuracy: {} %".format(test_accuracy))
    d = {"costs": costs,
         "Y_pred_test": Y_pred_test, 
         "Y_pred_train" : Y_pred_train, 
         "w" : w, 
         "b" : b,
         "learning_rate" : learning_rate,
         "num_iters": num_iters}
    return d,train_accuracy,test_accuracy

In [510]:
#choose number=0, implement model without over-sampling
x_train,train_labels,x_test,test_labels = preprocessing()
d = model(x_train, train_labels, x_test, test_labels, batch=20000, num_iters=10, learning_rate=0.01, print_cost=True)

(60000, 784)
(784, 60000)
5923
Cost after iteration 1: 0.243237
Cost after iteration 2: 0.224521
Cost after iteration 3: 0.208458
Cost after iteration 4: 0.194739
Cost after iteration 5: 0.183031
Cost after iteration 6: 0.173027
Cost after iteration 7: 0.164451
Cost after iteration 8: 0.157065
Cost after iteration 9: 0.150670
Cost after iteration 10: 0.145103

train accuracy: 90.12833333333333 %
test accuracy: 90.2 %


In [511]:
#choose number=1, implement model with over-sampling
x_train,train_labels,x_test,test_labels = preprocessing(flag=1,oversampling=True)
d = model(x_train, train_labels, x_test, test_labels, num_iters=10,batch=20000, learning_rate=0.01, print_cost=True)

(108154, 784)
(784, 108154)
54077
Cost after iteration 1: 0.245001
Cost after iteration 2: 0.233978
Cost after iteration 3: 0.224441
Cost after iteration 4: 0.215875
Cost after iteration 5: 0.208325
Cost after iteration 6: 0.201174
Cost after iteration 7: 0.194799
Cost after iteration 8: 0.188901
Cost after iteration 9: 0.183290
Cost after iteration 10: 0.178016

train accuracy: 50.0 %
test accuracy: 90.2 %
